In [1]:
import argparse
import time
import torch
from Models import get_model
from Process import *
import torch.nn.functional as F
from Optim import CosineWithRestarts
from Batch import create_masks
import pdb
import dill as pickle
import argparse
from Models import get_model
from Beam import beam_search
from nltk.corpus import wordnet
from torch.autograd import Variable
import re
from Batch import nopeak_mask
import math
import numpy as np
from pathlib import Path

In [2]:
%env CUDA_DEVICE_ORDER=PCI_BUS_ID
%env CUDA_VISIBLE_DEVICES=3


env: CUDA_DEVICE_ORDER=PCI_BUS_ID
env: CUDA_VISIBLE_DEVICES=3


In [3]:
parser = argparse.ArgumentParser()
#If we are working on small dataset
small = 0
#If we want to activate relevance based training
relevance_training = 1

if small==1:
    parser.add_argument('-src_data', type=str, default='data/italian_small.txt')
    parser.add_argument('-trg_data', type=str, default='data/english_small.txt')
    parser.add_argument('-trg_data_retrieval', type=str, default='data/english_retrieval.txt')

else:
    parser.add_argument('-src_data', type=str, default='data/italian.txt')
    parser.add_argument('-trg_data', type=str, default='data/english.txt')  
    parser.add_argument('-trg_data_retrieval', type=str, default='data/LATIMESTEXT2.txt')

parser.add_argument('-src_lang', type=str, default='it')
parser.add_argument('-trg_lang', type=str, default='en')
parser.add_argument('-no_cuda', action='store_true')
parser.add_argument('-SGDR', action='store_true')
parser.add_argument('-epochs', type=int, default=2)
parser.add_argument('-d_model', type=int, default=200)
parser.add_argument('-n_layers', type=int, default=6)
parser.add_argument('-heads', type=int, default=8)
parser.add_argument('-dropout', type=int, default=0.1)
parser.add_argument('-batchsize', type=int, default=1500)
parser.add_argument('-printevery', type=int, default=100)
parser.add_argument('-lr', type=int, default=0.0001)
parser.add_argument('-load_weights', type=str, default='weights')
parser.add_argument('-load_vocab', type=str, default='clir_it_en')
#parser.add_argument('-load_weights', type=str, default='tiny_train')
parser.add_argument('-create_valset', action='store_true')
parser.add_argument('-max_strlen', type=int, default=80)
parser.add_argument('-floyd', action='store_true')
parser.add_argument('-checkpoint', type=int, default=0)

#-load_weights clir_it_en -src_lang it -trg_lang en -k 2
#opt = parser.parse_args()
opt = parser.parse_args(args=[])
opt.device = 0 if opt.no_cuda is False else -1
print(opt)
#assert opt.k > 0
#assert opt.max_len > 10

Namespace(SGDR=False, batchsize=1500, checkpoint=0, create_valset=False, d_model=200, device=0, dropout=0.1, epochs=2, floyd=False, heads=8, load_vocab='clir_it_en', load_weights='weights', lr=0.0001, max_strlen=80, n_layers=6, no_cuda=False, printevery=100, src_data='data/italian.txt', src_lang='it', trg_data='data/english.txt', trg_data_retrieval='data/LATIMESTEXT2.txt', trg_lang='en')


In [4]:
def tokenizer(text):  # create a tokenizer function
        return text.split()

def create_fields(opt):    
    print("loading tokenizers...") 
    TRG = data.Field(lower=True, tokenize=tokenizer, init_token='<sos>', eos_token='<eos>')
    SRC = data.Field(lower=True, tokenize=tokenizer)   
    SRC = pickle.load(open(f'{opt.load_vocab}/SRC.pkl', 'rb'))
    TRG = pickle.load(open(f'{opt.load_vocab}/TRG.pkl', 'rb'))
    return(SRC, TRG)

def create_dataset(opt, SRC, TRG):

    print("creating dataset and iterator... ")

    raw_data = {'src' : [line for line in open(opt.src_data)], 'trg': [line for line in open(opt.trg_data)]}
    df = pd.DataFrame(raw_data, columns=["src", "trg"])

    mask = (df['src'].str.count(' ') < opt.max_strlen) & (df['trg'].str.count(' ') < opt.max_strlen)
    df = df.loc[mask]

    df.to_csv("translate_transformer_temp.csv", index=False)

    data_fields = [('src', SRC), ('trg', TRG)]
    train = data.TabularDataset('./translate_transformer_temp.csv', format='csv', fields=data_fields)

    train_iter = MyIterator(train, batch_size=opt.batchsize, device=opt.device,
                        repeat=False, sort_key=lambda x: (len(x.src), len(x.trg)),
                        batch_size_fn=batch_size_fn, train=True, shuffle=True)

    os.remove('translate_transformer_temp.csv')

    print("creating target vocabulary ... ")
    raw_data = {'trg': [line for line in open(opt.trg_data_retrieval)]}
    df = pd.DataFrame(raw_data, columns=["trg"])
    mask = (df['trg'].str.count(' ') > 1)
    df = df.loc[mask]
    df.to_csv("translate_transformer_retrieval_temp.csv", index=False)
    data_fields = [('trg', TRG)]
    train_retrieval = data.TabularDataset('./translate_transformer_retrieval_temp.csv', format='csv', fields=data_fields)
    os.remove('translate_transformer_retrieval_temp.csv')
    SRC.build_vocab(train)
    TRG.build_vocab(train, train_retrieval)
    #TRG.build_vocab(train_temp)
    if opt.checkpoint > 0:
        try:
            os.mkdir("weights")
        except:
            print("weights folder already exists, run program with -load_weights weights to load them")
            quit()
        pickle.dump(SRC, open('weights/SRC.pkl', 'wb'))
        pickle.dump(TRG, open('weights/TRG.pkl', 'wb'))
    opt.src_pad = SRC.vocab.stoi['<pad>']
    opt.trg_pad = TRG.vocab.stoi['<pad>']
    opt.train_len = get_len(train_iter)
    return train_iter

print(opt)
SRC, TRG = create_fields(opt)
#opt.train = create_dataset(opt, SRC, TRG)


Namespace(SGDR=False, batchsize=1500, checkpoint=0, create_valset=False, d_model=200, device=0, dropout=0.1, epochs=2, floyd=False, heads=8, load_vocab='clir_it_en', load_weights='weights', lr=0.0001, max_strlen=80, n_layers=6, no_cuda=False, printevery=100, src_data='data/italian.txt', src_lang='it', trg_data='data/english.txt', trg_data_retrieval='data/LATIMESTEXT2.txt', trg_lang='en')
loading tokenizers...


In [5]:
from Models import Transformer

model = Transformer(len(SRC.vocab), len(TRG.vocab), opt.d_model, opt.n_layers, opt.heads, opt.dropout)
model = model.cuda()
model.load_state_dict(torch.load(f'{opt.load_weights}/model_weights'))
#model.load_state_dict(torch.load(f'{opt.load_weights}/model_weights_1_epoch_large'))
#model = get_model(opt, len(SRC.vocab), len(TRG.vocab))
opt.k=3
opt.max_len=20
def get_synonym(word, SRC):
    syns = wordnet.synsets(word)
    for s in syns:
        for l in s.lemmas():
            if SRC.vocab.stoi[l.name()] != 0:
                return SRC.vocab.stoi[l.name()]

    return 0


def multiple_replace(dict, text):
    # Create a regular expression  from the dictionary keys
    regex = re.compile("(%s)" % "|".join(map(re.escape, dict.keys())))

    # For each match, look-up corresponding value in dictionary
    return regex.sub(lambda mo: dict[mo.string[mo.start():mo.end()]], text)

def preprocess_sentence(sentence, model, opt, SRC, TRG):
    indexed = []
    sentence = SRC.preprocess(sentence)
    for tok in sentence:
        if SRC.vocab.stoi[tok] != 0 or opt.floyd == True:
            indexed.append(SRC.vocab.stoi[tok])
        else:
            indexed.append(get_synonym(tok, SRC))
    sentence = Variable(torch.LongTensor([indexed]))

def translate_sentence(sentence, model, opt, SRC, TRG):
    model.eval()
    indexed = []
    sentence = SRC.preprocess(sentence)
    for tok in sentence:
        if SRC.vocab.stoi[tok] != 0 or opt.floyd == True:
            indexed.append(SRC.vocab.stoi[tok])
        else:
            indexed.append(get_synonym(tok, SRC))
    sentence = Variable(torch.LongTensor([indexed]))
    if opt.device == 0:
        sentence = sentence.cuda()

    sentences, query = beam_search(sentence, model, SRC, TRG, opt)
    #print(sentences)
    #print(query)
    
    for sentence in sentences: 
        multiple_replace({' ?': '?', ' !': '!', ' .': '.', '\' ': '\'', ' ,': ','}, sentence)
    return sentences, query


def translate(opt, model, SRC, TRG):
    sentences = opt.text.lower().split('.')
    translated = []
    queries = []
    #print(translate_sentence(sentence + '.', model, opt, SRC, TRG))
    for sentence in sentences:
        translated_sentences, query = translate_sentence(sentence + '.', model, opt, SRC, TRG)
        for translated_sentence in translated_sentences:
            translated.append(translated_sentence.capitalize())
        queries.append(query)

    return ('\n'.join(translated)), queries



In [6]:
def init_vars(src, model, SRC, TRG, opt):
    init_tok = TRG.vocab.stoi['<sos>']
    src_mask = (src != SRC.vocab.stoi['<pad>']).unsqueeze(-2)
    #this is the output from the encoder 
    e_output = model.encoder(src, src_mask)
    #this is initializing the outputs 
    outputs = torch.LongTensor([[init_tok]])
    if opt.device == 0:
        outputs = outputs.cuda()
    
    trg_mask = nopeak_mask(1, opt)
    src_mask = src_mask.cuda()
    trg_mask = trg_mask.cuda()
    outputs = outputs.cuda()
    e_output = e_output.cuda()

    out = model.out(model.decoder(outputs,
                                  e_output, src_mask, trg_mask))
    out = F.softmax(out, dim=-1)

    probs, ix = out[:, -1].data.topk(opt.k)
    preds_token_ids = ix.view(ix.size(0), -1)
    pred_strings = [' '.join([TRG.vocab.itos[ind] for ind in ex]) for ex in preds_token_ids]
    print (pred_strings)
    
    log_scores = torch.Tensor([math.log(prob) for prob in probs.data[0]]).unsqueeze(0)

    outputs = torch.zeros(opt.k, opt.max_len).long()
    if opt.device == 0:
        outputs = outputs.cuda()
    outputs[:, 0] = init_tok
    outputs[:, 1] = ix[0]

    e_outputs = torch.zeros(opt.k, e_output.size(-2), e_output.size(-1))
    if opt.device == 0:
        e_outputs = e_outputs.cuda()
    e_outputs[:, :] = e_output[0]

    return outputs, e_outputs, log_scores

def k_best_outputs(outputs, out, log_scores, i, k):
    probs, ix = out[:, -1].data.topk(k)
    preds_token_ids = ix.view(ix.size(0), -1)
    pred_strings = [' '.join([TRG.vocab.itos[ind] for ind in ex]) for ex in preds_token_ids]
    print (pred_strings)
    
    #print("indices of top k")
    #print(ix)
    log_probs = torch.Tensor([math.log(p) for p in probs.data.view(-1)]).view(k, -1) + log_scores.transpose(0, 1)
    k_probs, k_ix = log_probs.view(-1).topk(k)

    row = k_ix // k
    col = k_ix % k

    outputs[:, :i] = outputs[row, :i]
    outputs[:, i] = ix[row, col]

    log_scores = k_probs.unsqueeze(0)

    return outputs, log_scores, pred_strings


def beam_search(src, model, SRC, TRG, opt):
    outputs, e_outputs, log_scores = init_vars(src, model, SRC, TRG, opt)
    eos_tok = TRG.vocab.stoi['<eos>']
    src_mask = (src != SRC.vocab.stoi['<pad>']).unsqueeze(-2)
    ind = None
    query = {}
    for i in range(2, opt.max_len):

        trg_mask = nopeak_mask(i, opt)
        src_mask = src_mask.cuda()
        trg_mask = trg_mask.cuda()

        out = model.out(model.decoder(outputs[:, :i], e_outputs, src_mask, trg_mask))

        out = F.softmax(out, dim=-1)

        # print("output data shape")
        # print(out.data.shape)

        outputs, log_scores, pred_strings = k_best_outputs(outputs, out, log_scores, i, opt.k)
        
        for pred_string in pred_strings: 
            pred_string_splitted = pred_string.split()
            for st in pred_string_splitted:
                query.setdefault(st, 1.0)
                query[st] = query[st] + 1
        
        if (outputs == eos_tok).nonzero().size(0) == opt.k:
            alpha = 0.7
            div = 1 / ((outputs == eos_tok).nonzero()[:, 1].type_as(log_scores) ** alpha)
            _, ind = torch.max(log_scores * div, 1)
            ind = ind.data[0]
            break
    #print("query")
    #print(query)
    # if ind is None:
    #     length = (outputs[0] == eos_tok).nonzero()[0]
    #     return ' '.join([TRG.vocab.itos[tok] for tok in outputs[0][1:length]])
    #
    # else:
    #     length = (outputs[ind] == eos_tok).nonzero()[0]
    # return ' '.join([TRG.vocab.itos[tok] for tok in outputs[ind][1:length]])

    if ind is None:
        query_list = []
        #print("value of k is " + str(opt.k))
        for i in np.arange(opt.k):
            length = (outputs[i] == eos_tok).nonzero()[0]
            #length = opt.max_len
            query_list.append(' '.join([TRG.vocab.itos[tok] for tok in outputs[i][1:length]]))
        return query_list, query

        # if (outputs[0]==eos_tok).nonzero().size(0) >= 1:
        #     length = (outputs[0]==eos_tok).nonzero()[0]
        #     return ' '.join([TRG.vocab.itos[tok] for tok in outputs[0][1:length]])
        # else:
        #     return ' '


    else:
        # if (outputs[ind] == eos_tok).nonzero().size(0) >= 1:
        #     length = (outputs[ind]==eos_tok).nonzero()[0]
        #     return ' '.join([TRG.vocab.itos[tok] for tok in outputs[ind][1:length]])
        # else:
        #     return ' '
        query_list = []
        #print("value of k is " + str(opt.k))
        for i in np.arange(opt.k):
            length = (outputs[i]==eos_tok).nonzero()[0]
            #length = opt.max_len
            query_list.append(' '.join([TRG.vocab.itos[tok] for tok in outputs[i][1:length]]))
        return query_list, query


In [7]:
print(type(SRC))

<class 'torchtext.data.field.Field'>


In [8]:
print(type(TRG))

<class 'torchtext.data.field.Field'>


In [9]:
opt.text = "epidemia ebola zaire recupera documenti parlano misure preventive prese dopo scoppio epidemia ebola zaire"
#opt.text = "german parlamento documenti artista christo"
#opt.text = "trova documenti parlano impacchettamento parlamento tedesco berlino opera artista christo"

In [10]:
phrase, queries = translate(opt, model, SRC, TRG)


def create_galago_query(tuple_list):
    st="#combine:"
    
    for index, tuple in enumerate(tuple_list):
        st+= str(index) + "=" + str(tuple[1])
        st+=":"
    st = st.rstrip(":") 
    st+="("
    for index, tuple in enumerate(tuple_list):
         st+= str(tuple[0] + " ")
    st+=")"
    print(st)
    
import pandas as pd
import operator
for query in queries:
    sum = 0.0
    for token in query.keys():
        sum+=query[token]
    for token in query.keys():
        query[token]/=sum
    sorted_query = sorted(query.items(), key=operator.itemgetter(1))
    print (sorted_query)
    create_galago_query(sorted_query)
    #print(sorted(queries, lambda key = queries[1]))

['epidemic outbreak famine']
['zaire famine devastated', 'zaire mozambique saudi', 'zaire devastated mozambique']
['recover returning getting', 'returning recover will', 'zaire mozambique saudi']
['will get getting', 'documents papers reports', 'documents papers reports']
['talk speak talking', 'talk talking speak', 'get acquire pass']
['preventive prevention measures', 'preventive prevention measures', 'preventive prevention measures']
['measures action steps', 'measures action steps', 'measures action steps']
['taken took adopted', 'taken took outbreak', 'taken took adopted']
['outbreak epidemic disease', 'outbreak epidemic disease', 'outbreak epidemic disease']
['epidemic outbreak disease', 'epidemic outbreak disease', 'epidemic outbreak disease']
['caused epidemic outbreak', 'caused epidemic outbreak', 'epidemic outbreak disease']
['<eos> sherlock delaware', '<eos> disease epidemic', '<eos> caused devastated']
['<eos> disease weapons', '<eos> weapons base', '<eos> elaborating sherl

In [11]:
print(TRG.vocab.itos[4880])
print(TRG.vocab.itos[1283])
print(TRG.vocab.itos[1977])

menu
rise
prove


In [12]:
print(phrase)

Outbreak zaire recover documents talk preventive measures taken outbreak outbreak outbreak
Outbreak zaire recover documents talk preventive measures taken outbreak outbreak epidemic
Outbreak zaire recover documents talk preventive measures taken outbreak outbreak outbreak
